# Dependencies

In [1]:
import pandas as pd
import numpy as np
from musixmatch import Musixmatch
from API_Key import api_key3
import time
from pprint import pprint

In [2]:
import sqlalchemy
from sqlalchemy import create_engine, inspect, Column, Integer, Float, String, Sequence, Boolean
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base

In [3]:
musixmatch = Musixmatch(api_key3)

# Setting up SQLlite

In [4]:
#Using declarative base
Base = declarative_base()
engine = create_engine('sqlite:///lyrics3.db', echo = False)

In [6]:
#defining the Tracks table
class Tracks(Base):
    __tablename__ = "tracks"
    
    id = Column(Integer, primary_key=True, nullable = False, unique = True, autoincrement = True)
    album_id = Column(Integer)
    artist_id = Column(Integer)
    artist_name = Column(String)
    explicit = Column(Integer)
    has_lyrics = Column(Integer)
    lyrics_id = Column(Integer)
    track_id = Column(Integer, unique = True)
    track_name = Column(String)

In [7]:
#defining the Lyrics table
class Lyrics(Base):
    __tablename__ = "lyrics"
    
    id = Column(Integer, primary_key=True, nullable = False, unique = True, autoincrement = True)
    explicit = Column(Integer)
    lyrics_body = Column(String)
    lyrics_id = Column(Integer, unique = True)
    lyrics_language = Column(String)
    lyrics_language_description = Column(String)

In [8]:
#defining the Artists table
class Artists(Base):
    __tablename__ = "artists"
    
    id = Column(Integer, primary_key=True, nullable = False, unique = True, autoincrement = True)
    artist_id = Column(Integer, unique = True)
    artist_name = Column(String)
    artist_q = Column(String)

In [9]:
#defining the Albums table
class Albums(Base):
    __tablename__ = "albums"
    
    id = Column(Integer, primary_key=True, nullable = False, unique = True, autoincrement = True)
    artist_id = Column(Integer)
    artist_name = Column(String)
    album_id = Column(Integer, unique = True)
    album_name = Column(String)

In [6]:
#defining the genres table
class Album_Genres(Base):
    __tablename__ = "album_genres"
    
    id = Column(Integer, primary_key=True, nullable = False, unique = True, autoincrement = True)
    artist_id = Column(Integer)
    artist_name = Column(String)
    album_id = Column(Integer)
    genre = Column(String)
    genre_id = Column(Integer)

In [5]:
#Creating the table metadata
Base.metadata.create_all(engine)
session = Session(engine)

In [6]:
#Binding engine to the metadata
meta = sqlalchemy.MetaData()
meta.reflect(bind = engine)

# MusixMatch API - Finding artists via text search from manually created list

In [11]:
artists_path = 'artist_names.xlsx'
artists_df = pd.read_excel(artists_path, sheetname = "Sheet1", header = None)
artists = list(artists_df[0].values)

In [12]:
#Connecting to tracks table
table = sqlalchemy.Table('artists', meta, autoload = True)

In [ ]:
for artist in artists:
    print(f"getting artist names {artist}")
    result = musixmatch.artist_search(q_artist = artist, page = 1, page_size = 100, f_artist_id = '', f_artist_mbid = '')
    print('extracting results')
    artist_list = result['message']['body']['artist_list']
    artists = [artist['artist'] for artist in artist_list]
    print("creating package")
    for item in artists:
        package = {
            'artist_id': item['artist_id'],
            'artist_name': item['artist_name'],
            'artist_q': artist
        }
        try:
            print("writing package to database")
            session.execute(table.insert(), package)
        except:
            next
    print("committing to database")
    session.commit()

# MusixMatch API - Finding Albums from Artists

In [26]:
artists = pd.read_sql('artists', engine)
artists.dropna(inplace = True)
artist_ids = list(artists['artist_id'].values)

In [12]:
table = sqlalchemy.Table('albums', meta, autoload = True)

In [ ]:
for artist_id in artist_ids:
    print(f"getting albums for artist {artist_id}")
    result = musixmatch.artist_albums_get(artist_id = artist_id, g_album_name = '', page = 1, page_size = 100, s_release_date = '')
    print("extracting the album_list")
    album_list = result['message']['body']['album_list']
    print("extracting individual albums")
    albums = [album['album'] for album in album_list]
    print("iterating through the list of albums")
    for item in albums:
        print("creating the package")
        package = {
            'album_id': item['album_id'],
            'album_name': item['album_name'],
            'artist_id': item['artist_id'],
            'artist_name': item['artist_name']
        }
        try:
            print("writing to the database")
            session.execute(table.insert(), package)
        except:
            next
    print("comitting to the database")
    session.commit()

# MusixMatch API - Scraping Tracks From Charts

In [19]:
#Connecting to tracks table
table = sqlalchemy.Table('tracks', meta, autoload = True)

In [10]:
country_codes = ['AU', 'CA', 'IE', 'IM', 'NZ', 'GB', 'US']

In [ ]:
country_count = 1
for country in country_codes:
    
    print(f"Attempting country {country_count} for {country}")
    
    page_count = 1

    while page_count < 5:
        print(f"calling api attempt {page_count} for {country}")
        result = musixmatch.chart_tracks_get(page_count, 100, f_has_lyrics = True)
        tracks = result['message']['body']['track_list']
        track_list = [track['track'] for track in tracks]
        print(f"track list is {len(track_list)}")
        for track in track_list:
            package = {
                'album_id': track['album_id'],
                'artist_id': track['artist_id'],
                'artist_name': track['artist_name'],
                'explicit': track['explicit'],
                'has_lyrics': track['has_lyrics'],
                'lyrics_id': track['lyrics_id'],
                'track_id': track['track_id'],
                'track_name': track['track_name']
            }
            print(package['track_name'])
            print("writing to database")
            try:
                session.execute(table.insert(), package)
            except:
                next
        page_count += 1
        time.sleep(1)
        session.commit()
    
    country_count += 1

# MusixMatch API - finding tracks from albums

## Starting point is the tracks data from the charts

In [20]:
tracks = pd.read_sql_table('tracks', engine, index_col = "id", coerce_float = False)

In [21]:
album_ids = tracks['album_id'].values

In [24]:
#Connecting to tracks table
table = sqlalchemy.Table('tracks', meta, autoload = True)

In [ ]:
album_count = 1

for album_id in album_ids:
    print(f"getting result {album_count} for {album_id}")
    result = musixmatch.album_tracks_get(album_id = album_id, album_mbid = "", page = 1, page_size = 100)
    print(f"getting track_list for {album_id}")
    album = result['message']['body']['track_list']
    print("creating track list")
    track_list = [track['track'] for track in album]
    track_count = 1
    print(f"Iterating through tracks in album")
    for track in track_list:
        print(f"track is {track['track_id']}")
        package = {
                'album_id': int(track['album_id']),
                'artist_id': int(track['artist_id']),
                'artist_name': track['artist_name'],
                'explicit': int(track['explicit']),
                'has_lyrics': int(track['has_lyrics']),
                'lyrics_id': int(track['lyrics_id']),
                'track_id': int(track['track_id']),
                'track_name': track['track_name']
            }
        print(package['track_name'])
        print("writing to database")
        try:
            session.execute(table.insert(), package)
        except:
            print("writing to database failed")
            next
        print("committing files to database")
        session.commit()
    album_count += 1

## Starting point is the album's data

In [15]:
albums = pd.read_sql_table('albums', engine, index_col = "id", coerce_float = False)
albums = albums.iloc[647:]
album_ids = list(albums['album_id'].values)

In [16]:
#Connecting to tracks table
table = sqlalchemy.Table('tracks', meta, autoload = True)

In [ ]:
album_count = 1

for album_id in album_ids:
    print(f"getting result {album_count} for {album_id}")
    result = musixmatch.album_tracks_get(album_id = album_id, album_mbid = "", page = 1, page_size = 100)
    print(f"getting track_list for {album_id}")
    album = result['message']['body']['track_list']
    print("creating track list")
    track_list = [track['track'] for track in album]
    track_count = 1
    print(f"Iterating through tracks in album")
    for track in track_list:
        print(f"track is {track['track_id']}")
        package = {
                'album_id': int(track['album_id']),
                'artist_id': int(track['artist_id']),
                'artist_name': track['artist_name'],
                'explicit': int(track['explicit']),
                'has_lyrics': int(track['has_lyrics']),
                'lyrics_id': int(track['lyrics_id']),
                'track_id': int(track['track_id']),
                'track_name': track['track_name']
            }
        print(package['track_name'])
        print("writing to database")
        try:
            session.execute(table.insert(), package)
        except:
            print("writing to database failed")
            next
        print("committing files to database")
        session.commit()
    album_count += 1

## MusixMatch API - Scraping Lyrics from the Albums Data

In [28]:
#Connecting to lyrics table
table = sqlalchemy.Table('lyrics', meta, autoload = True)

In [29]:
tracks = pd.read_sql('tracks', engine, index_col = "id")
track_ids = tracks['track_id'].values

In [ ]:
count = 1
for track in track_ids:
    print(f"calling API attempt {count} for track_id# {track}")
    print(f"track data type is {type(track)}")
    lyrics = musixmatch.track_lyrics_get(track)
    print("pulling out lyrics package")
    lyrics_package = lyrics['message']['body']['lyrics']
    print("pulling out package to insert into SQL database")
    package = {
        "explicit": lyrics_package['explicit'],
        "lyrics_body": lyrics_package["lyrics_body"][:-69],
        "lyrics_id": lyrics_package['lyrics_id'],
        "lyrics_language": lyrics_package['lyrics_language'],
        "lyrics_language_description": lyrics_package['lyrics_language_description'],
        "updated_time": lyrics_package['updated_time']
    }
    
    print(package['lyrics_body'][:10])
    print("writing to database")
    try:
        session.execute(table.insert(), package)
    except:
        next
    print("committing files to database")
    session.commit()
    count += 1 

# Acquiring Genre from Album

In [7]:
#Connecting to lyrics table
table = sqlalchemy.Table('album_genres', meta, autoload = True)

In [8]:
albums = pd.read_sql_table('tracks', engine, index_col = "id", coerce_float = False)
album_ids = list(albums['album_id'].values)

In [9]:
len(album_ids)

27742

In [10]:
album_array = np.array(album_ids)

In [11]:
album_unique = np.unique(album_array)
len(album_unique)

3398

In [12]:
genre_dic = {}

In [13]:
x = 0
for id in album_unique[0:2000]:
    
    result = musixmatch.album_get(album_id=id)
    album = result['message']['body']['album']['album_name']
    artist = result['message']['body']['album']['artist_name']
    print(f'album id is {id}')
    print(f'the album is {album}')
    print(f'the artist is {artist}')
    genre=''
    try:
        genre = result['message']['body']['album']['primary_genres']['music_genre_list'][0]['music_genre']['music_genre_name']
        #genre_list = [genre['music_genre'] for genre in genres]
        print(f'the album genre is {genre}')
    except IndexError:
        pass
    try:
        genre = result['message']['body']['album']['secondary_genres']['music_genre_list'][0]['music_genre']['music_genre_name']
        print(f'the album genre is {genre}')
    except IndexError:
        pass
    results_dic = {"artist": artist, "album": album, "genre": genre}
    genre_dic[f'{id}'] = results_dic
    print(f'---------- completed {x} of 1600')
    x+=1
#genre_dic['War']

album id is 10266031
the album is Big Willie Style
the artist is Will Smith
---------- completed 0 of 1600
album id is 10266041
the album is War
the artist is U2
the album genre is Rock
---------- completed 1 of 1600
album id is 10266127
the album is Led Zeppelin
the artist is Led Zeppelin
---------- completed 2 of 1600
album id is 10266180
the album is Power Windows
the artist is Rush
---------- completed 3 of 1600
album id is 10266202
the album is Willennium
the artist is Will Smith
---------- completed 4 of 1600
album id is 10266231
the album is Parachutes
the artist is Coldplay
---------- completed 5 of 1600
album id is 10266299
the album is Hold Your Fire
the artist is Rush
the album genre is Rock
---------- completed 6 of 1600
album id is 10266444
the album is Counterparts
the artist is Rush
---------- completed 7 of 1600
album id is 10267122
the album is 1956-1958: Cobra Recordings
the artist is Otis Rush
---------- completed 8 of 1600
album id is 10267496
the album is The Cobra

album id is 10290929
the album is Third Eye Open: The String Quartet Tribute to Tool
the artist is Vitamin String Quartet
---------- completed 80 of 1600
album id is 10291870
the album is - 1958
the artist is Otis Rush
---------- completed 81 of 1600
album id is 10291909
the album is Blues, Songs and Ballads
the artist is Tom Rush
---------- completed 82 of 1600
album id is 10292180
the album is Undisputed Attitude
the artist is Slayer
the album genre is Rock
---------- completed 83 of 1600
album id is 10292199
the album is Code Red
the artist is DJ Jazzy Jeff & The Fresh Prince
---------- completed 84 of 1600
album id is 10294108
the album is Metallica
the artist is Metallica
---------- completed 85 of 1600
album id is 10294187
the album is Heart Over Mind
the artist is Jennifer Rush
---------- completed 86 of 1600
album id is 10294194
the album is Jennifer Rush
the artist is Jennifer Rush
the album genre is Rock
---------- completed 87 of 1600
album id is 10295104
the album is Big Wi

album id is 10333185
the album is Greatest Hits
the artist is DJ Jazzy Jeff & The Fresh Prince
---------- completed 156 of 1600
album id is 10333681
the album is Big Ones
the artist is Aerosmith
---------- completed 157 of 1600
album id is 10334791
the album is Hell Awaits
the artist is Slayer
---------- completed 158 of 1600
album id is 10335664
the album is Rush
the artist is Darude
---------- completed 159 of 1600
album id is 10335742
the album is Next to You
the artist is Darude
---------- completed 160 of 1600
album id is 10336088
the album is Psychoanalysis: What Is It?
the artist is Prince Paul
---------- completed 161 of 1600
album id is 10338312
the album is Load
the artist is Metallica
---------- completed 162 of 1600
album id is 10338419
the album is And In This Corner...
the artist is DJ Jazzy Jeff & The Fresh Prince
the album genre is Hip Hop/Rap
---------- completed 163 of 1600
album id is 10338602
the album is St. Anger
the artist is Metallica
---------- completed 164 of

album id is 10410136
the album is Peace Love Death Metal
the artist is Eagles of Death Metal
---------- completed 231 of 1600
album id is 10411452
the album is Western Music
the artist is Will Oldham
---------- completed 232 of 1600
album id is 10411457
the album is The Whitey on the Moon UK LP
the artist is Department of Eagles
---------- completed 233 of 1600
album id is 10411460
the album is Black/Rich Music
the artist is Will Oldham
---------- completed 234 of 1600
album id is 10411468
the album is Happy Child / Forest Time
the artist is Bonnie "Prince" Billy
the album genre is Alternative
---------- completed 235 of 1600
album id is 10411497
the album is Hush Up!!!
the artist is Prince Buster
---------- completed 236 of 1600
album id is 10412323
the album is Double Trouble
the artist is Otis Rush
---------- completed 237 of 1600
album id is 10412407
the album is Black Dissimulation
the artist is Bonnie "Prince" Billy
---------- completed 238 of 1600
album id is 10412410
the album 

album id is 10496307
the album is The Everlasting Love (New and Best)
the artist is U2
---------- completed 307 of 1600
album id is 10499287
the album is Sandstorm
the artist is Darude
---------- completed 308 of 1600
album id is 10499507
the album is No More Workhorse Blues
the artist is Bonnie "Prince" Billy
the album genre is Alternative
---------- completed 309 of 1600
album id is 10503008
the album is The Hits/The B-Sides
the artist is Prince
---------- completed 310 of 1600
album id is 10504480
the album is Lost and Found
the artist is Will Smith
---------- completed 311 of 1600
album id is 10509748
the album is Switch
the artist is Will Smith
---------- completed 312 of 1600
album id is 10509865
the album is Any Place I'm Going
the artist is Otis Rush
---------- completed 313 of 1600
album id is 10511855
the album is Make Believe
the artist is Weezer
---------- completed 314 of 1600
album id is 10512229
the album is Trolling for Owls
the artist is Tom Rush
---------- completed 3

...

album id is 15507672
the album is Psalms for I
the artist is Prince Far I
the album genre is Reggae
the album genre is World
---------- completed 1711 of 1600
album id is 15509960
the album is Bonnie "Prince" Billy
the artist is Bonnie "Prince" Billy
---------- completed 1712 of 1600
album id is 15510096
the album is Music from Another Dimension!
the artist is Aerosmith
---------- completed 1713 of 1600
album id is 15511228
the album is Maladroit
the artist is Weezer
---------- completed 1714 of 1600
album id is 15514959
the album is Lose Yourself
the artist is Eminem
the album genre is Hip Hop/Rap
---------- completed 1715 of 1600
album id is 15514963
the album is Curtain Call - The Hits
the artist is Eminem
the album genre is Hip Hop/Rap
---------- completed 1716 of 1600
album id is 15514971
the album is Relapse
the artist is Eminem
the album genre is Hip Hop/Rap
---------- completed 1717 of 1600
album id is 15514973
the album is Encore
the artist is Eminem
the album genre is Hip Hop

album id is 15601680
the album is Phase II
the artist is Prince Royce
the album genre is Salsa y Tropical
---------- completed 1766 of 1600
album id is 15608349
the album is Heart Over Mind
the artist is Jennifer Rush
the album genre is Pop
---------- completed 1767 of 1600
album id is 15610407
the album is Big Shiny Tunes 13
the artist is Buckcherry, Coldplay, Die Mannequin, Disturbed, LINKIN PARK, Panic! At the Disco, Protest the Hero, Queens of the Stone Age, Sam Roberts, Saving Abel, Seether, The Kooks, The Mission District, The Raconteurs, The Verve, Tokyo Police Club, U.S.S, Weezer & Wintersleep
the album genre is Rock
---------- completed 1768 of 1600
album id is 15614470
the album is My Name Is Prince
the artist is Prince & The New Power Generation
---------- completed 1769 of 1600
album id is 15617379
the album is Will Smith: Greatest Hits
the artist is Will Smith
the album genre is Pop
---------- completed 1770 of 1600
album id is 15617381
the album is Will Smith: Greatest Hi

album id is 15714250
the album is Long Road Out of Eden
the artist is Eagles
the album genre is Rock
the album genre is Country
---------- completed 1828 of 1600
album id is 15714258
the album is Please Come Home for Christmas / Funky New Year
the artist is Eagles
the album genre is Holiday
---------- completed 1829 of 1600
album id is 15714721
the album is Fast Lane
the artist is Bad Meets Evil
the album genre is Hip Hop/Rap
---------- completed 1830 of 1600
album id is 15717642
the album is Otis Rush: Live at Montreux 1986
the artist is Otis Rush
the album genre is Blues
---------- completed 1831 of 1600
album id is 15718958
the album is Reign In Blood
the artist is Slayer
the album genre is Rock
---------- completed 1832 of 1600
album id is 15718961
the album is God Hates Us All
the artist is Slayer
the album genre is Rock
---------- completed 1833 of 1600
album id is 15718965
the album is Christ Illusion
the artist is Slayer
the album genre is Heavy Metal
---------- completed 1834 

album id is 15946709
the album is The Psychedelic Years (1966-1968)
the artist is The Revolution
the album genre is Rock
---------- completed 1891 of 1600
album id is 15947111
the album is Label This!
the artist is Darude
the album genre is Pop
---------- completed 1892 of 1600
album id is 15948606
the album is The Revolution: The Singles Album
the artist is The Revolution
the album genre is Rock
---------- completed 1893 of 1600
album id is 15953296
the album is Voices In My Head
the artist is Leo Sayer
the album genre is Pop
---------- completed 1894 of 1600
album id is 15955874
the album is Lost and Found
the artist is Will Smith
the album genre is R&B/Soul
---------- completed 1895 of 1600
album id is 15957317
the album is Leo Sayer: Live In 1975
the artist is Leo Sayer
the album genre is Pop
---------- completed 1896 of 1600
album id is 15959490
the album is Endless Journey – The Essential Leo Sayer
the artist is Leo Sayer
the album genre is Pop
the album genre is Vocal
----------

album id is 16169189
the album is The Hit Box
the artist is Jennifer Rush
the album genre is Pop
---------- completed 1952 of 1600
album id is 16169190
the album is Jennifer Rush: Superhits
the artist is Jennifer Rush
the album genre is Pop
---------- completed 1953 of 1600
album id is 16169191
the album is Jennifer Rush: Hit Collection
the artist is Jennifer Rush
the album genre is Pop
---------- completed 1954 of 1600
album id is 16169192
the album is The Very Best of (Her EMI / Virgin Years)
the artist is Jennifer Rush
the album genre is Pop
the album genre is Vocal
---------- completed 1955 of 1600
album id is 16169193
the album is Jennifer Rush: Best of 1983-2010
the artist is Jennifer Rush
the album genre is Pop
---------- completed 1956 of 1600
album id is 16169194
the album is The Power of Jennifer Rush
the artist is Jennifer Rush
the album genre is Pop
---------- completed 1957 of 1600
album id is 16169195
the album is Essential Jennifer Rush
the artist is Jennifer Rush
the al

In [14]:
total_df = pd.DataFrame.from_dict(genre_dic, orient='index')

In [16]:
total_df.head()

,artist,album,genre
10266031,Will Smith,Big Willie Style,
10266041,U2,War,Rock
10266127,Led Zeppelin,Led Zeppelin,
10266180,Rush,Power Windows,
10266202,Will Smith,Willennium,


In [18]:
total_df.to_csv('scrape_test_data.csv')

In [36]:
from bs4 import BeautifulSoup
from splinter import Browser
from selenium import webdriver
import requests

driver = webdriver.Chrome()

In [43]:
genres = []
for i in range(10):
    artist = total_df['artist'][i]
    album = total_df['album'][i]
    web = 'https://www.allmusic.com/search/albums/'
    
    #Create a splinter of the webpage to start scraping
    executable_path = {"executable_path": "resources/chromedriver.exe"}
    browser = Browser('chrome', **executable_path, headless=False)
    
    url = web + artist + " " + album
    
    browser.visit(url)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    genre = soup.find("div", class_='genres').get_text()
    print(f"{artist}'s album, {album}, genre is {genre}")
    driver = webdriver.Chrome()
    browser.Close()

Will Smith's album, Big Willie Style, genre is 
                Rap            


AttributeError: 'WebDriver' object has no attribute 'Close'

In [22]:
total_df['artist'][0]

'Will Smith'

In [ ]:
## THIS IS FOR TESTING ONLY
import pprint
data = musixmatch.album_get(album_id='10295432')
pprint.pprint(data)

In [ ]:
album_count = 1

for album_id in album_unique:
    print(f"getting result {album_count} for {album_id}")
    result = musixmatch.album_get(album_id = album_id)
    print(f"getting information for {album_id}")
    album = result['message']['body']['album']
    print("creating genre list")
    genres = result['message']['body']['album']['primary_genres']['music_genre_list']
    genre_list = [genre['music_genre'] for genre in genres]
    print(f"Iterating through album genres")
    for genre in track_list:
        package = {
                'album_id': int(album['album_id']),
                'artist_id': int(album['artist_id']),
                'artist_name': album['artist_name'],
                'genre': genre['music_genre_name'],
                'genre_id': genre['music_genre_id']
                
            }
        print(package['album_id'])
        print("writing to database")
        try:
            session.execute(table.insert(), package)
        except:
            print("writing to database failed")
            next
        print("committing files to database")
        session.commit()
    album_count += 1